In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from sklearn.metrics import accuracy_score

In [ ]:
plt.style.use("ggplot")

In [ ]:
drive.mount("content/drive")

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/mnist/train.csv")
test = pd.read_csv("/content/drive/MyDrive/mnist/test.csv")

In [ ]:
X_train = train.drop(columns="label").values
y_train = train["label"].values

In [ ]:
X_test = test.values

In [ ]:
# Normalize the data
X_train = X_train / 255
X_test = X_test / 255

In [ ]:
# Reshape to image format
X_train = X_train.reshape(-1, 1, 28, 28)
X_test = X_test.reshape(-1, 1, 28, 28)

In [ ]:
# One hot encoding
y_train_oh = torch.eye(10)[y_train]

In [ ]:
# Convert X, y to tensor
X_train = torch.Tensor(X_train)
y_train_oh = torch.Tensor(y_train_oh)

X_test = torch.Tensor(X_test)

In [ ]:
class CNN(nn.module):
    def __init__(self):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1) 
        self.relu2 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout1 = nn.Dropout(0.3)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.relu3 = nn.ReLU()

        self.conv4 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.relu4 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout2 = nn.Dropout(0.3)

        self.conv5 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.relu5 = nn.ReLU()
        
        self.conv6 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.relu6 = nn.ReLU()
        self.dropout3 = nn.Dropout(0.35)

        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(256 * 7 * 7, 512)
        self.relu7 = nn.ReLU()
        self.dropout4 = nn.Dropout(0.5)

        self.fc2 = nn.Linear(512, 10)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool1(x)
        x = self.dropout1(x)

        x = self.conv3(x)
        x = self.relu3(x)

        x = self.conv4(x)
        x = self.relu4(x)
        x = self.pool2(x)
        x = self.dropout2(x)

        x = self.conv5(x)
        x = self.relu5(x)

        x = self.conv6(x)
        x = self.relu6(x)
        x = self.dropout3(x)

        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu7(x)
        x = self.dropout4(x)

        x = self.fc2(x)
        x = self.softmax(x)

        return x

In [ ]:
num_samples = X_train.shape[0]

In [ ]:
num_filters1 = 8
num_filters2 = 16
num_hidden_units=256
learning_rate = 0.001
num_epochs = 10
batch_size = 250

model = CNN()

# Move the model, input data, and labels to the GPU
model = model.to(device)
X_train = X_train.to(device)
y_train_oh = y_train_oh.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    for start_index in range(0, num_samples, batch_size):
        end_index = start_index + batch_size
        if end_index > num_samples: continue
        batch = X_train[start_index:end_index]

        # Forward pass
        y_train_oh_ = model(X_train)
        loss = criterion(y_train_oh_, y_train_oh)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [ ]:
y_train_oh_ = model(X_train)
y_train_ = torch.argmax(y_train_oh_, dim=1).cpu().numpy()

accuracy = accuracy_score(y_train, y_train_)

print(f"Training accuracy: {accuracy}")

In [ ]:
# Move X_test to GPU
X_test = X_test.to(device)

# Make predictions from X_test
y_test_oh_ = model(X_test)
y_test_ = torch.argmax(y_test_oh_, dim=1).cpu().numpy()

In [ ]:
test_predictions = pd.Series(y_test_, name="Label")
test_predictions.index.name = "ImageId"
test_predictions.index += 1 # kaggle expects one indexed predictions

In [ ]:
test_predictions.to_csv("/content/drive/MyDrive/mnist/cnn_baseline2.csv")